# Comparador de JSONs
Aquest notebook té com a objectiu fer un anàlisi profund dels diferents formats de JSON que les diferents màquines presenten. 
El notebook conté una classe que gestiona els JSONs a partir dels seus csv's originals, i els emmagatzema a un diccionari per a poder iterar i estudiar-lo fàcilment. 


# 0. Import de llibreries i Setup

In [ ]:
import os
import csv
import glob
import json
import pandas as pd
from tqdm import tqdm
from pprint import pprint

from collections import defaultdict


# print current os location
print("Current working directory:", os.getcwd())

Current working directory: c:\Users\AlexMalo\Desktop\Servimatic\servimatic\Servimatic


# 1. Gestió del csv amb totes les dades

## 1.1 Concatenació dels CSV en el Master

In [ ]:
full_dataset_path = os.path.join(os.path.dirname(os.getcwd()), 'data','MensajesRAW', 'Mensajes_5.csv')

columns = [
    "ID", "MsgID", "FechaMensaje", "ReqID", "SgID", "Maquina", "SGVer",
    "ConexionTipoID", "MensajeTipoID", "Mensaje", "FechaModificacion", "UsuarioModificacion"
]
full_df= pd.read_csv(full_dataset_path, sep=';', names = columns, header=None)
full_df.head()

,ID,MsgID,FechaMensaje,ReqID,SgID,Maquina,SGVer,ConexionTipoID,MensajeTipoID,Mensaje,FechaModificacion,UsuarioModificacion
0,104289216,173,2025-10-19 00:07:36.000,0,237747333746909,MRE2.2092.22,1.12.13-37,1,25,"{""TotalAccount"": {""Bet"": 22461240, ""Win"": 1623...",2025-10-19 00:08:23.000,golondrina
1,104289431,1132,2025-10-19 00:10:13.000,0,237747333853080,NLS2.7216.25,1.12.13-37,1,25,"{""TotalAccount"": {""Bet"": 28630, ""Win"": 19880, ...",2025-10-19 00:11:13.770,golondrina
2,104289503,1134,2025-10-19 00:11:13.000,0,237747333853080,NLS2.7216.25,1.12.13-37,1,25,"{""TotalAccount"": {""Bet"": 28630, ""Win"": 19880, ...",2025-10-19 00:12:13.910,golondrina
3,104289570,1136,2025-10-19 00:12:13.000,0,237747333853080,NLS2.7216.25,1.12.13-37,1,25,"{""TotalAccount"": {""Bet"": 28630, ""Win"": 19880, ...",2025-10-19 00:13:14.090,golondrina
4,104289653,1138,2025-10-19 00:13:13.000,0,237747333853080,NLS2.7216.25,1.12.13-37,1,25,"{""TotalAccount"": {""Bet"": 30630, ""Win"": 19880, ...",2025-10-19 00:14:14.223,golondrina


In [ ]:
all_files = sorted(glob.glob(os.path.join(os.path.dirname(os.getcwd()), 'data', 'MensajesRAW', 'Mensajes_*.csv')))

full_df = pd.concat(
    (pd.read_csv(f, sep=';', names=columns, header=None) for f in all_files),
    ignore_index=True
)

print(f"✅ DataFrame creat amb {len(full_df)} files i {len(full_df.columns)} columnes")

# convert full_df to csv and save it in data folder
full_df.to_csv(os.path.join(os.path.dirname(os.getcwd()), 'data', 'master_mensajes.csv'), index=False)

✅ DataFrame creat amb 2662888 files i 12 columnes


## 1.2 Petit EDA del csv Master 

In [42]:
full_df.head()

,ID,MsgID,FechaMensaje,ReqID,SgID,Maquina,SGVer,ConexionTipoID,MensajeTipoID,Mensaje,FechaModificacion,UsuarioModificacion
0,102013728,849,2025-10-06 00:01:42.000,0,237747332448291,MRE3.383.24,1.12.14-37,1,25,"{""TotalAccount"": {""Bet"": 5448480, ""Win"": 43342...",2025-10-06 02:01:56.740,golondrina
1,102013763,7,2025-10-06 00:03:34.000,57540,237747333837472,NGO2.21847.24,1.12.14-37,1,19,"{""SystemInfo"": {""Revision"": ""b03115"", ""Serial""...",2025-10-06 02:03:36.790,golondrina
2,102013764,9,2025-10-06 00:03:34.000,57544,237747333837472,NGO2.21847.24,1.12.14-37,1,68,"{""Log"": {""2025-10-05"": {""TotalAccount"": {""Bet""...",2025-10-06 02:03:36.910,golondrina
3,102013765,10,2025-10-06 00:03:34.000,57545,237747333837472,NGO2.21847.24,1.12.14-37,1,67,"{""Status"": ""OK""}",2025-10-06 02:03:36.983,golondrina
4,102013766,11,2025-10-06 00:03:34.000,57546,237747333837472,NGO2.21847.24,1.12.14-37,1,69,"{""Status"": ""OK""}",2025-10-06 02:03:37.073,golondrina


In [43]:
print("DataFrame Statistics:")
print(f"Number of rows: {full_df.shape[0]}")
print(f"Number of columns: {full_df.shape[1]}")
print(f"Columns: {full_df.columns.tolist()}")
print(f"Missing values per column:\n{full_df.isnull().sum()}")
print(f"Data types:\n{full_df.dtypes}")

DataFrame Statistics:
Number of rows: 2662888
Number of columns: 12
Columns: ['ID', 'MsgID', 'FechaMensaje', 'ReqID', 'SgID', 'Maquina', 'SGVer', 'ConexionTipoID', 'MensajeTipoID', 'Mensaje', 'FechaModificacion', 'UsuarioModificacion']
Missing values per column:
ID                     0
MsgID                  0
FechaMensaje           0
ReqID                  0
SgID                   0
Maquina                0
SGVer                  0
ConexionTipoID         0
MensajeTipoID          0
Mensaje                0
FechaModificacion      0
UsuarioModificacion    0
dtype: int64
Data types:
ID                      int64
MsgID                   int64
FechaMensaje           object
ReqID                   int64
SgID                    int64
Maquina                object
SGVer                  object
ConexionTipoID          int64
MensajeTipoID           int64
Mensaje                object
FechaModificacion      object
UsuarioModificacion    object
dtype: object


In [ ]:
machines = full_df['Maquina'].unique()
print(f"Number of unique machines: {len(machines)}\n")

messages_per_machine = full_df['Maquina'].value_counts()
print(f"Minimum number of messages per machine: {messages_per_machine.min()}")  
print(f"Maximum number of messages per machine: {messages_per_machine.max()}")
print(f"Average number of messages per machine: {messages_per_machine.mean()}")
print(f"Median number of messages per machine: {messages_per_machine.median()}")
print(f"25th percentile number of messages per machine: {messages_per_machine.quantile(0.25)}")
print(f"75th percentile number of messages per machine: {messages_per_machine.quantile(0.75)}\n")

# Also give info on how many machines have under 84, 168 and 336 messages (corresponding to 1, 2 and 4 weeks of messages if we assume 12 messages per day).
thresholds = [84, 168, 336]
for threshold in thresholds:
    count = (messages_per_machine < threshold).sum()
    print(f"Number of machines with under {threshold} messages: {count}")



Number of unique machines: 2501

Minimum number of messages per machine: 3
Maximum number of messages per machine: 21021
Average number of messages per machine: 1064.7293082766894
Median number of messages per machine: 707.0
25th percentile number of messages per machine: 293.0
75th percentile number of messages per machine: 1421.0

Number of machines with under 84 messages: 174
Number of machines with under 168 messages: 345
Number of machines with under 336 messages: 690



Les estadístiques sobre el número de missatges per màquina son les següents:

<table style="border-collapse: collapse; width: 80%; text-align: center;">
  <thead>
    <tr style="background-color: #444; color: white;">
      <th style="border: 1px solid #ddd; padding: 8px;">Mínim Registres</th>
      <th style="border: 1px solid #ddd; padding: 8px;">Màxim Registres</th>
      <th style="border: 1px solid #ddd; padding: 8px;">Mitja</th>
      <th style="border: 1px solid #ddd; padding: 8px;">STD</th>
      <th style="border: 1px solid #ddd; padding: 8px;">Num Màquines</th>
      <th style="border: 1px solid #ddd; padding: 8px;">Q1</th>
      <th style="border: 1px solid #ddd; padding: 8px;">Mitjana</th>
      <th style="border: 1px solid #ddd; padding: 8px;">Q3</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td style="border: 1px solid #ddd; padding: 8px;">3</td>
      <td style="border: 1px solid #ddd; padding: 8px;">21021</td>
      <td style="border: 1px solid #ddd; padding: 8px;">1057</td>
      <td style="border: 1px solid #ddd; padding: 8px;">1326</td>
      <td style="border: 1px solid #ddd; padding: 8px;">2501</td>
      <td style="border: 1px solid #ddd; padding: 8px;">293</td>
      <td style="border: 1px solid #ddd; padding: 8px;">707</td>
      <td style="border: 1px solid #ddd; padding: 8px;">1421</td>
    </tr>
  </tbody>
</table>

Considerant que tenim dues setmanes de data i aproximant que els bars obren ~12h al dia, si una màquina envia un missatge per hora, donaria un total de 168 missatges. 

També podem veure una taula amb els valors de numeros de taules que tenen menys de X missatges:

<table style="border-collapse: collapse; width: 80%; text-align: center;">
  <thead>
    <tr style="background-color: #db1717ff; color: white;">
      <th style="border: 1px solid #ddd; padding: 8px;"> >84 Missatges </th>
      <th style="border: 1px solid #ddd; padding: 8px;">>168 Missatges</th>
      <th style="border: 1px solid #ddd; padding: 8px;">>336 Missatges</th>
    </tr>
  </thead> 
  <tbody>
    <tr>
      <td style="border: 1px solid #ddd; padding: 8px;">174 Màquines</td>
      <td style="border: 1px solid #ddd; padding: 8px;">345 Màquines </td>
      <td style="border: 1px solid #ddd; padding: 8px;">690 Màquines</td>
    </tr>
  </tbody>
</table> 

In [ ]:
full_df = pd.read_csv(os.path.join(os.path.dirname(os.getcwd()), 'data', 'master_mensajes.csv'))

#### Inspecció dels tipus de missatge que tenim

In [11]:
full_df['MensajeTipoID'].value_counts()

MensajeTipoID
25    2450281
40      48229
19      41132
68      41016
67      41004
69      41004
28        131
64         91
Name: count, dtype: int64

Recordem que el missatge de tipus 25 és el que ens interessa, i.e el que conté tota la informació sobre el Bet i Win. 

Vegem els altres tipus de missatges que tenim

In [24]:
# for each MensajeTipoID, select and show one sample message from full_df
for mensaje_tipo_id in full_df['MensajeTipoID'].unique():
    if mensaje_tipo_id == 25:
        continue
    sample_message1 = full_df[full_df['MensajeTipoID'] == mensaje_tipo_id]['Mensaje'].iloc[0]
    sample_message2 = full_df[full_df['MensajeTipoID'] == mensaje_tipo_id]['Mensaje'].iloc[15]
    sample_message3 = full_df[full_df['MensajeTipoID'] == mensaje_tipo_id]['Mensaje'].iloc[68]
    sample_message4 = full_df[full_df['MensajeTipoID'] == mensaje_tipo_id]['Mensaje'].iloc[90]

    print(f"MensajeTipoID: {mensaje_tipo_id}\nSample Message 1: {sample_message1}\nSample Message 2: {sample_message2}\nSample Message 3: {sample_message3}\nSample Message 4: {sample_message4}\n")

MensajeTipoID: 19
Sample Message 1: {"SystemInfo": {"Revision": "b03115", "Serial": "1000000044f71c75", "EthMac": "d8:3a:dd:61:32:a0", "WlanMac": "d8:3a:dd:61:32:a1", "BtMac": "D8:3A:DD:61:32:A2", "Model": "4", "Os": 10, "SGId": "237747333837472", "SGVer": "1.12.14-37", "Rele": true, "Hat": 15, "PwdVer": 1}, "Status": {"ReleStatus": false, "CPUTemp": 55.0, "Debug": false, "ComBaudRate": 4800, "BtStatus": true, "ConnectionType": 1, "MeterValue": 20, "TestMode": false, "DiagMode": false, "Wifi": {"Config": {"ssid": "Moll Vell", "scan_ssid": "1", "psk": "lm9T55WMMP9qtA=="}, "LinkTo": "Moll Vell", "RSSI": 5}, "Modem": {}, "DefaultProtocol": {"Id": 5, "Name": "SAS", "Version": "", "Running": false, "LastError": "", "Denominacion": 0}, "ExtraProtocol": {}}}
Sample Message 2: {"SystemInfo": {"Revision": "a020d3", "Serial": "8cf55ec4", "EthMac": "b8:27:eb:f5:5e:c4", "WlanMac": "b8:27:eb:a0:0b:91", "BtMac": "B8:27:EB:5F:F4:6E", "Model": "3", "Os": 10, "SGId": "202481601961668", "SGVer": "1.12.1

## 1.3 Separació del CSV Master en màquines

Creem funció que agafa el fitxer csv amb totes les dades i els separa, creant un petit fitxer per a cada màquina. 


In [ ]:
# Old version (no buffering)
columns= [
    "ID", "MsgID", "FechaMensaje", "ReqID", "SgID", "Maquina", "SGVer",
    "ConexionTipoID", "MensajeTipoID", "Mensaje", "FechaModificacion", "UsuarioModificacion"
]
def split_csv_by_machine(input_file_path):
    output_dir = os.path.join(os.path.dirname(input_file_path), 'Missatges')
    os.makedirs(output_dir, exist_ok=True)

    with open(input_file_path, 'r', encoding='utf-8', errors='ignore') as infile:
        reader = csv.reader(infile, delimiter=';')

        for row in reader:
            if len(row) < 6:  # Skip malformed rows
                continue

            machine_id = row[5].strip()
            if not machine_id:
                continue

            machine_file_path = os.path.join(output_dir, f'{machine_id}.csv')

            # Append mode to avoid keeping files open
            write_header = not os.path.exists(machine_file_path)
            with open(machine_file_path, 'a', encoding='utf-8', newline='') as outfile:
                writer = csv.writer(outfile, delimiter=';')
                if write_header:
                    writer.writerow(columns)
                writer.writerow(row)



Versió optimitzada amb buffer per processar el csv més ràpidament

In [ ]:

def split_csv_by_machine_buffered(input_file_path, buffer_size=500):
    output_dir = os.path.join(os.path.dirname(input_file_path), 'Missatges')
    os.makedirs(output_dir, exist_ok=True)

    buffers = {}  # {machine_id: [rows]}
    
    def flush_buffer(machine_id):
        """Write buffered rows to the corresponding file and clear buffer."""
        machine_file_path = os.path.join(output_dir, f'{machine_id}.csv')
        write_header = not os.path.exists(machine_file_path)
        with open(machine_file_path, 'a', encoding='utf-8', newline='') as outfile:
            writer = csv.writer(outfile, delimiter=';')
            if write_header:
                writer.writerow(columns)
            writer.writerows(buffers[machine_id])
        buffers[machine_id] = []

    with open(input_file_path, 'r', encoding='utf-8', errors='ignore') as infile:
        reader = csv.reader(infile, delimiter=';')
        
        for row in reader:
            if len(row) < 6:
                continue
            machine_id = row[5].strip()
            if not machine_id:
                continue

            if machine_id not in buffers:
                buffers[machine_id] = []

            buffers[machine_id].append(row)

            # Flush buffer if it reaches buffer_size
            if len(buffers[machine_id]) >= buffer_size:
                flush_buffer(machine_id)

    # Flush remaining buffers
    for machine_id in buffers:
        if buffers[machine_id]:
            flush_buffer(machine_id)

    print("✅ Split completed successfully!")

In [ ]:

columns= [
            "ID", "MsgID", "FechaMensaje", "ReqID", "SgID", "Maquina", "SGVer",
            "ConexionTipoID", "MensajeTipoID", "Mensaje", "FechaModificacion", "UsuarioModificacion"
        ]

def split_csv_by_machine_buffered(input_file_path, buffer_size=500):
    output_dir = os.path.join(os.path.dirname(input_file_path), 'Missatges_x_maquina')
    os.makedirs(output_dir, exist_ok=True)

    buffers = {}  # {machine_id: [rows]}

    def flush_buffer(machine_id):
        """Write buffered rows to the corresponding file and clear buffer."""
        machine_file_path = os.path.join(output_dir, f'{machine_id}.csv')
        write_header = not os.path.exists(machine_file_path)
        with open(machine_file_path, 'a', encoding='utf-8', newline='') as outfile:
            writer = csv.writer(outfile, delimiter=',')
            if write_header:
                writer.writerow(columns)
            writer.writerows(buffers[machine_id])
        buffers[machine_id] = []

    # Count total lines for progress bar
    total_lines = sum(1 for _ in open(input_file_path, 'r', encoding='utf-8', errors='ignore')) - 1  # minus header

    with open(input_file_path, 'r', encoding='utf-8', errors='ignore') as infile:
        reader = csv.reader(infile, delimiter=',')
        next(reader, None)  # Skip header row

        for row in tqdm(reader, total=total_lines, desc="Processing rows"):
            if len(row) < 6:
                continue
            machine_id = row[5].strip()
            if not machine_id:
                continue

            if machine_id not in buffers:
                buffers[machine_id] = []

            buffers[machine_id].append(row)

            # Flush buffer if it reaches buffer_size
            if len(buffers[machine_id]) >= buffer_size:
                flush_buffer(machine_id)

    # Flush remaining buffers
    for machine_id in buffers:
        if buffers[machine_id]:
            flush_buffer(machine_id)

    print("✅ Split completed successfully!")


In [9]:
full_dataset_path = os.path.join(os.path.dirname(os.getcwd()), 'data', 'master_mensajes.csv')
split_csv_by_machine_buffered(full_dataset_path)

Processing rows: 100%|██████████| 2662888/2662888 [04:43<00:00, 9400.29it/s] 


✅ Split completed successfully!


Comprovació inicial de que els arxius s'han creat bé, obrint un aleatori i fent petita inspecció

In [16]:
random_dataset = os.path.join(os.path.dirname(os.getcwd()), 'data', 'Missatges_x_maquina', '7PO2.2034.25.csv')
df_random = pd.read_csv(random_dataset)
df_random.head()

,ID,MsgID,FechaMensaje,ReqID,SgID,Maquina,SGVer,ConexionTipoID,MensajeTipoID,Mensaje,FechaModificacion,UsuarioModificacion
0,102498406,7,2025-10-08 14:37:22.000,56058,237747332448199,7PO2.2034.25,1.12.14-37,1,19,"{""SystemInfo"": {""Revision"": ""b03115"", ""Serial""...",2025-10-08 16:37:22.007,golondrina
1,102498407,9,2025-10-08 14:37:23.000,56062,237747332448199,7PO2.2034.25,1.12.14-37,1,68,"{""Log"": {}}",2025-10-08 16:37:22.080,golondrina
2,102498408,10,2025-10-08 14:37:23.000,56063,237747332448199,7PO2.2034.25,1.12.14-37,1,67,"{""Status"": ""OK""}",2025-10-08 16:37:22.140,golondrina
3,102498410,11,2025-10-08 14:37:23.000,56064,237747332448199,7PO2.2034.25,1.12.14-37,1,69,"{""Status"": ""OK""}",2025-10-08 16:37:22.207,golondrina
4,102498576,12,2025-10-08 14:38:18.000,0,237747332448199,7PO2.2034.25,1.12.14-37,1,25,"{""TotalAccount"": {""Bet"": 0, ""Win"": 0, ""Games"":...",2025-10-08 16:38:17.557,golondrina


## 1.3 Eliminació de petits CSV



In [ ]:
def remove_small_machine_files(output_dir, min_rows=85):
    """
    Elimina els arxius CSV que tinguin menys de min_rows (inclosa el header).
    """
    for file_name in os.listdir(output_dir):
        if file_name.endswith('.csv'):
            file_path = os.path.join(output_dir, file_name)
            with open(file_path, 'r', encoding='utf-8') as f:
                row_count = sum(1 for _ in f) - 1  # Restamos la cabecera
            if row_count < min_rows:
                os.remove(file_path)
                print(f"❌ Eliminado: {file_name} ({row_count} registros)")
            else:
                print(f"✅ Conservado: {file_name} ({row_count} registros)")


In [ ]:
output_dir = os.path.join(os.path.dirname(os.getcwd()), 'data', 'Missatges')
remove_small_machine_files(output_dir, min_rows=85)

# 2. Anàlisi dels diferents JSON

Creem una classe que ens permeti analitzar els diferents tipus de JSON per als missatges del tipus 25. Separem aquesta classe en diferents funcions per a assegurar escalabilitat en la que tenim flexibilitat per afegir noves funcionalitats, o analitzar només algun fitxer concret, i ens assegurem un debugging més senzill. A més, la classe queda molt més senzilla de llegir i entendre.

<h2>📘 Manual d'ús: <code>AnalitzadorEstructuresJSON</code></h2>


<h3>Objectius principals:</h3>
<ul>
  <li>Detectar les <strong>estructures úniques</strong> dels JSONs.</li>
  <li>Guardar <strong>quins fitxers tenen cada estructura</strong>.</li>
  <li>Permetre <strong>consultar</strong> sobre claus principals i derivades.</li>
</ul>

<hr>

<h3>⚙️ Inicialització</h3>

<pre><code>analitzador = AnalitzadorEstructuresJSON("ruta/a/la/carpeta")</code></pre>

<ul>
  <li><code>carpeta_csv</code>: ruta on es troben els fitxers <code>.csv</code> amb els missatges.</li>
</ul>

<hr>

<h3>🔍 Mètodes disponibles</h3>

<h4><code>analitzar_fitxers()</code></h4>
<p>Analitza tots els fitxers CSV de la carpeta especificada. Per cada fitxer:</p>
<ul>
  <li>Busca el primer missatge amb <code>MensajeTipoID == 25</code>.</li>
  <li>Extreu l'estructura del JSON.</li>
  <li>Guarda l'estructura si és nova.</li>
  <li>Assigna el fitxer a la seva estructura corresponent.</li>
</ul>

<hr>

<h4><code>mostrar_estructures_uniques()</code></h4>
<p>Mostra totes les estructures úniques trobades. Per cada estructura:</p>
<ul>
  <li>Llista les claus principals i les claus derivades.</li>
  <li>Indica <strong>quants fitxers</strong> tenen aquesta estructura.</li>
</ul>

<pre>
🧩 Estructura 1 (152 fitxers):
  PartAccount: Bet, Win, CreditsIn, ...
  GamesBet: Value, Games
</pre>

<hr>

<h4><code>buscar_claus_derivades(clau_principal)</code></h4>
<p>Permet consultar les <strong>claus derivades</strong> d'una clau principal en totes les estructures.</p>

<pre><code>analitzador.buscar_claus_derivades("PartAccount")</code></pre>

<pre>
🧩 Estructura 1: Bet, Win, CreditsIn, ...
🧩 Estructura 2: ❌ Clau no trobada
</pre>

<hr>

<h4><code>mostrar_json_maquina(nom_fitxer)</code></h4>
<p> Mostra l'estructura JSON (claus derivades) d'una màquina concreta. L'input és el nom del fitxer sense l'extensió .csv.</p>

<hr>

<h2>📦 Atributs interns</h2>

<h4><code>self.estructures_uniques</code></h4>
<p>Llista de totes les estructures úniques trobades. Cada estructura és un diccionari com:</p>

<pre><code>{
    "PartAccount": ["Bet", "Win", "CreditsIn", ...],
    "GamesBet": ["Value", "Games"],
    ...
}</code></pre>

<h4><code>self.estructures_fitxers</code></h4>
<p>Diccionari que relaciona cada estructura amb els fitxers que la tenen:</p>

<pre><code>{
    estructura_hashable: ["maquina_001.csv", "maquina_002.csv", ...]
}</code></pre>




In [ ]:
class AnalitzadorEstructuresJSON:
    def __init__(self, carpeta_csv):
        self.carpeta_csv = carpeta_csv
        self.estructures_uniques = []  # Llista de mapes de claus derivades
        self.estructures_fitxers = defaultdict(list)  # Mapa: estructura_hashable -> llista de fitxers

    def analitzar_fitxers(self):
        """
        Analitza tots els fitxers CSV a la carpeta especificada i guarda estructures úniques.
        """
        for nom_fitxer in tqdm(os.listdir(self.carpeta_csv), desc="Analitzant fitxers CSV"):
            if nom_fitxer.endswith(".csv"):
                ruta_fitxer = os.path.join(self.carpeta_csv, nom_fitxer)
                self._analitzar_fitxer(ruta_fitxer)


    def _analitzar_fitxer(self, ruta_fitxer):
        """
        Analitza un fitxer CSV per extreure l'estructura JSON dels missatges de tipus 25.
        """
        try:
            df = pd.read_csv(ruta_fitxer)
            df_25 = df[df["MensajeTipoID"] == 25]

            if not df_25.empty:
                missatge_json = df_25.iloc[0]["Mensaje"]
                estructura = self._mapa_claus_derivades(missatge_json)
                estructura_hashable = self._estructura_com_hash(estructura)

                if estructura_hashable not in self.estructures_fitxers:
                    self.estructures_uniques.append(estructura)

                self.estructures_fitxers[estructura_hashable].append(os.path.basename(ruta_fitxer))
        except Exception as e:
            print(f"Error analitzant {ruta_fitxer}: {e}")

    def _mapa_claus_derivades(self, missatge):
        """
        Construeix un mapa de claus derivades a partir del JSON.
        """
        mapa = defaultdict(set)

        def recorre(dades, pare=None):
            if isinstance(dades, dict):
                for k, v in dades.items():
                    if pare:
                        mapa[pare].add(k)
                    recorre(v, k)
            elif isinstance(dades, list):
                for element in dades:
                    recorre(element, pare)

        try:
            dades = json.loads(missatge)
            recorre(dades)
            return {k: sorted(list(v)) for k, v in mapa.items()}
        except json.JSONDecodeError:
            return {"Error": ["JSON invàlid"]}

    def _estructura_com_hash(self, estructura):
        """
        Converteix una estructura en una representació hashable (tuple de tuples).
        """
        return tuple(sorted((k, tuple(sorted(v))) for k, v in estructura.items()))

    def mostrar_estructures_uniques(self):
        """
        Mostra totes les estructures úniques trobades.
        """
        print(f"\n🔎 S'han trobat {len(self.estructures_uniques)} estructures úniques:\n")
        for i, estructura in enumerate(self.estructures_uniques, 1):
            estructura_hashable = self._estructura_com_hash(estructura)
            fitxers = self.estructures_fitxers[estructura_hashable]
            print(f"🧩 Estructura {i} ({len(fitxers)} fitxers):")
            for clau, derivades in estructura.items():
                print(f"  {clau}: {', '.join(derivades)}")
            print()

    def mostrar_fitxers_per_estructura(self):
        """
        Mostra quins fitxers pertanyen a cada estructura única.
        """
        print(f"\n📂 Fitxers agrupats per estructura:\n")
        for i, estructura in enumerate(self.estructures_uniques, 1):
            estructura_hashable = self._estructura_com_hash(estructura)
            fitxers = self.estructures_fitxers[estructura_hashable]
            print(f"🧩 Estructura {i} ({len(fitxers)} fitxers):")
            print(f"  Fitxers: {', '.join(fitxers)}\n")

    def buscar_claus_derivades(self, clau_principal):
        """
        Mostra les claus derivades d'una clau principal en totes les estructures úniques.
        """
        print(f"\n🔍 Claus derivades per a la clau '{clau_principal}':\n")
        for i, estructura in enumerate(self.estructures_uniques, 1):
            if clau_principal in estructura:
                derivades = estructura[clau_principal]
                print(f"🧩 Estructura {i}: {', '.join(derivades)}")
            else:
                print(f"🧩 Estructura {i}: ❌ Clau no trobada")

    
    def mostrar_json_maquina(self, nom_fitxer):
        """
        Mostra l'estructura JSON (claus derivades) d'una màquina concreta. L'input és el nom del fitxer sense l'extensió .csv.
        
        """
        try:
            ruta_fitxer = os.path.join(self.carpeta_csv, nom_fitxer+'.csv')
            df = pd.read_csv(ruta_fitxer)
            df_25 = df[df["MensajeTipoID"] == 25]

            if not df_25.empty:
                missatge_json = json.loads(df_25.iloc[0]["Mensaje"])               
                print(f"\n📄 Estructura JSON per a la màquina '{nom_fitxer}':\n")
                pprint(missatge_json, sort_dicts=True)
            else:
                print(f"⚠️ No s'ha trobat cap missatge de tipus 25 al fitxer '{nom_fitxer}'.")
        except Exception as e:
            print(f"❌ Error llegint el fitxer '{nom_fitxer}': {e}\nRecorda que el nom del fitxer NO ha d'incloure l'extensió .csv.")
            print("Exemple de input correcte: '7PO2.2034.25'")


Entrant 2501 fitxers, 2487 se'ls assigna a una de les estructures úniques. Per el mometn no s'ha investigat a fons què passa amb les altres. Si en algun moment aixó respresentés un perill, es podria fer sense dificultat. 

In [78]:
test = AnalitzadorEstructuresJSON(os.path.join(os.path.dirname(os.getcwd()), 'data', 'Missatges_x_maquina'))
test.analitzar_fitxers()
test.mostrar_estructures_uniques()

Analitzant fitxers CSV: 100%|██████████| 2501/2501 [02:57<00:00, 14.08it/s]


🔎 S'han trobat 14 estructures úniques:

🧩 Estructura 1 (7 fitxers):
  TotalAccount: Bet, DateMeters, Games, HandPay, LastTotalBet, LastTotalWin, MeterValue, TotalBet, TotalWin, Win
  TotalAccountDetail: GambleLose, GambleTimes, GambleWin, GamesBet
  GamesBet: Games, Value
  TotalCash: In, Out, Refill
  TotalCashDetail: CoinMeters, NoteMeters
  CoinMeters: In, Out, Refill, TestIn, TestOut, Value
  NoteMeters: In, Out, Refill, TestIn, TestOut, Value
  PartAccount: Bet, Games, HandPay, MeterEvents, ResetDateTime, TestBet, TestGames, TestHandPay, TestMode, TestWin, Win
  PartAccountDetail: GambleLose, GambleTimes, GambleWin, GamesBet
  PartCash: In, Out, Refill, TestIn, TestOut
  PartCashDetail: CoinMeters, NoteMeters
  MultiGameAccount: MultiGameMeters
  MultiSatAccount: MultiSatMeters
  TimeTables: TimeIdle, TimePlay
  WarningMeters: WarningsList
  PulseMeters: DateRecA, DateRec_A, MeterValue, PulseE, PulseEA, PulseEP, PulseEP_A, PulseE_A, PulseS, PulseSA, PulseSP, PulseSP_A, PulseS_A



In [79]:
test.mostrar_fitxers_per_estructura()


📂 Fitxers agrupats per estructura:

🧩 Estructura 1 (7 fitxers):
  Fitxers: 7PO2.2033.25.csv, BWL2.299.25.csv, GNX2.1285.13.csv, IMF2.511087.25.csv, MRE2.2705.23.csv, MRE6.395.24.csv, SG.237747340960545.csv

🧩 Estructura 2 (702 fitxers):
  Fitxers: 7PO2.2034.25.csv, 7PO2.2035.25.csv, 7PO2.2036.25.csv, 7PO2.2037.25.csv, 7PO2.2192.25.csv, 7PO3.1739.25.csv, 7PO3.1740.25.csv, 7PO3.1741.25.csv, 7PO3.1742.25.csv, 7PO3.1743.25.csv, ANG2.4369.16.csv, ASB2.93.17.csv, ASB6.301.18.csv, ASB6.324.18.csv, ASB7.10261.19.csv, ASB7.10270.19.csv, ASB7.10782.19.csv, ASB7.16.19.csv, ASB7.23.19.csv, ASB7.72.19.csv, ASB7.83.19.csv, BOD3.2699.23.csv, BOD5.1341.12.csv, BW82.1197.22.csv, BW82.1213.22.csv, BW82.133.24.csv, BW82.385.23.csv, BW82.386.23.csv, BW82.450.22.csv, BW82.456.22.csv, BW82.464.22.csv, BW82.470.22.csv, BW82.661.23.csv, BW82.662.23.csv, BW82.664.23.csv, BW82.665.23.csv, BW82.670.23.csv, BW82.672.23.csv, BW82.935.21.csv, BW82.948.21.csv, BW83.349.22.csv, BW83.353.22.csv, BWG2.1154.23.csv, BWG

In [82]:
test.buscar_claus_derivades('MultiGameMeters')


🔍 Claus derivades per a la clau 'MultiGameMeters':

🧩 Estructura 1: ❌ Clau no trobada
🧩 Estructura 2: ❌ Clau no trobada
🧩 Estructura 3: GameId, GameName, PartBet, PartGames, PartWin, TotalBet, TotalGames, TotalWin
🧩 Estructura 4: GameId, GameName, PartBet, PartGames, PartWin, TotalBet, TotalGames, TotalWin
🧩 Estructura 5: ❌ Clau no trobada
🧩 Estructura 6: ❌ Clau no trobada
🧩 Estructura 7: ❌ Clau no trobada
🧩 Estructura 8: ❌ Clau no trobada
🧩 Estructura 9: ❌ Clau no trobada
🧩 Estructura 10: ❌ Clau no trobada
🧩 Estructura 11: GameId, GameName, PartBet, PartGames, PartWin, TotalBet, TotalGames, TotalWin
🧩 Estructura 12: GameId, GameName, PartBet, PartGames, PartWin, TotalBet, TotalGames, TotalWin
🧩 Estructura 13: GameId, GameName, PartBet, PartGames, PartWin, TotalBet, TotalGames, TotalWin
🧩 Estructura 14: ❌ Clau no trobada


In [ ]:
test.mostrar_json_maquina('NPG3.133899.24')


📄 Estructura JSON per a la màquina 'NPG3.133899.24':

{'MultiGameAccount': {'MultiGameMeters': [{'GameId': 1,
                                           'GameName': 'Lord of the Ocean',
                                           'PartBet': 28353,
                                           'PartGames': 775,
                                           'PartWin': 42020,
                                           'TotalBet': 716913,
                                           'TotalGames': 24208,
                                           'TotalWin': 1259270},
                                          {'GameId': 2,
                                           'GameName': 'Book of Ra Deluxe',
                                           'PartBet': 40700,
                                           'PartGames': 1432,
                                           'PartWin': 68580,
                                           'TotalBet': 2066923,
                                           'TotalGames': 7